In [40]:
import pandas as pd
import numpy as np
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.tokenize import word_tokenize 
import re
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [41]:
#загрузка данных

df = pd.read_csv('train.csv')

In [42]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [43]:
stopwords_list = stopwords.words('russian')
for word in ['не', 'нет', 'да', 'никогда', 'без', 'хорошо', 'лучше']:
  stopwords_list.remove(word)

In [44]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [45]:
#обработка

def preprocess(string):
  word_tokens = word_tokenize(string.lower())
  return " ".join([lemmatizer.lemmatize(w) for w in word_tokens if not w in stopwords_list and w.isalpha()])

In [46]:
#jxbcnrf текста

df['sentence'] = df['sentence'].map(preprocess)

In [47]:
#gfp,bnbt на тренировочный и валидационный наборы данных

training_data, test_data = train_test_split(df, train_size = 0.7, random_state=42, stratify = df.sentiment)

In [48]:
#инициализация модели tf-idf

tfidf_transform = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), lowercase=True, max_features=30000)

In [49]:
X_train = tfidf_transform.fit_transform(training_data['sentence'])
X_validation = tfidf_transform.transform(test_data['sentence'])
y_train = training_data['sentiment']
y_test = test_data['sentiment']

In [50]:
logit = LogisticRegression()

In [51]:
#обучение

logit.fit(X_train, y_train)

/usr/local/lib/python3.9/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [52]:
#accuracy на валидационном датасете

logit.score(X_validation, y_test) 

0.8504045446720606

In [53]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test, logit.predict_proba(X_validation), multi_class='ovr')

0.9345566412618619